<a href="https://colab.research.google.com/github/Ajeet-18/classicalAI/blob/main/mini_max_human_vs_al.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Write a python code to implement the Mini-Max algorithm for the Tic-Tac-Toe game. This code should compute the minimax values of the entire game tree starting from the initial blank state. Write the minimax values to a json file (or any other suitable format) after the training. Think of techniques you can use to reduce the size of this file.

2. Write a python code that uses the output of #1 (from the json file) to play the game against a human player.

In [ ]:
from math import inf as infinity

In [ ]:
from random import choice

In [ ]:
import platform

In [ ]:
import time

In [ ]:
from os import system

In [ ]:
HUMAN = -1
AI = +1
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]


def evaluate(state):                                                      # evaluate state, +1 if the AI wins; -1 if the human wins; 0 draw
    if wins(state, AI):
        score = +1
    elif wins(state, HUMAN):
        score = -1
    else:
        score = 0

    return score

In [ ]:
def wins(state, player):                                                   # This function tests if a specific player wins
    win_state = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    if [player, player, player] in win_state:                              # return: True, if the player wins
        return True
    else:
        return False


def game_over(state):                                                      # This function test if the human or AI wins
    return wins(state, HUMAN) or wins(state, AI)                           # return: True, if the human or AI win


def empty_cells(state):                                                    # Each empty cell will be added into cells' list
    cells = []                                                             # return: a, list of empty cells

    for x, row in enumerate(state):
        for y, cell in enumerate(row):
            if cell == 0:
                cells.append([x, y])

    return cells


In [ ]:
def valid_move(x, y):                                                     # A move is valid if the chosen cell is empty
    if [x, y] in empty_cells(board):                                       # return: True if the board[x][y] is empty
        return True
    else:
        return False

In [ ]:
def set_move(x, y, player):                                               # Set the move on board, if the coordinates are valid
    if valid_move(x, y):
        board[x][y] = player
        return True
    else:
        return False

In [ ]:
def minimax(state, depth, player):                                        # AI function that choice the best move
    if player == AI:                                                      # return: a list with [the best row, best col, best score
        best = [-1, -1, -infinity]
    else:
        best = [-1, -1, +infinity]

    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [-1, -1, score]

    for cell in empty_cells(state):
        x, y = cell[0], cell[1]
        state[x][y] = player
        score = minimax(state, depth - 1, -player)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == AI:
            if score[2] > best[2]:
                best = score                                            # max value
        else:
            if score[2] < best[2]:
                best = score                                            # min value

    return best

In [ ]:
def clean():                                                            # Clears the console
    os_name = platform.system().lower()
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')

In [ ]:
def render(state, c_choice, h_choice):                                   # Print the board on consol
    chars = {
        -1: h_choice,
        +1: c_choice,
        0: ' '
    }
    str_line = '---------------'

    print('\n' + str_line)
    for row in state:
        for cell in row:
            symbol = chars[cell]
            print(f'| {symbol} |', end='')
        print('\n' + str_line)

In [ ]:
def ai_turn(c_choice, h_choice):                                          # It calls the minimax function if the depth < 9,else it choices a random coordinate
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    clean()
    print(f'AI turn [{c_choice}]')
    render(board, c_choice, h_choice)

    if depth == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(board, depth, AI)
        x, y = move[0], move[1]

    set_move(x, y, AI)
    time.sleep(1)

In [ ]:
def human_turn(c_choice, h_choice):                                     # Human plays choosing a valid move.
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    # Dictionary of valid moves
    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clean()
    print(f'Human turn [{h_choice}]')
    render(board, c_choice, h_choice)

    while move < 1 or move > 9:
        try:
            move = int(input('Use numpad (1..9): '))
            coord = moves[move]
            can_move = set_move(coord[0], coord[1], HUMAN)

            if not can_move:
                print('Bad move')
                move = -1
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

In [ ]:
def main():                                                              # Main function that calls all functions
    clean()
    h_choice = ''  # X or O
    c_choice = ''  # X or O
    first = ''  # if human is the first

    # Human chooses X or O to play
    while h_choice != 'O' and h_choice != 'X':
        try:
            print('')
            h_choice = input('Choose X or O\nChosen: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Setting computer's choice
    if h_choice == 'X':
        c_choice = 'O'
    else:
        c_choice = 'X'

    # Human may starts first
    clean()
    while first != 'Y' and first != 'N':
        try:
            first = input('First to start?[y/n]: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Main loop of this game
    while len(empty_cells(board)) > 0 and not game_over(board):
        if first == 'N':
            ai_turn(c_choice, h_choice)
            first = ''

        human_turn(c_choice, h_choice)
        ai_turn(c_choice, h_choice)

    # Game over message
    if wins(board, HUMAN):
        clean()
        print(f'Human turn [{h_choice}]')
        render(board, c_choice, h_choice)
        print('YOU WIN!')
    elif wins(board, AI):
        clean()
        print(f'AI turn [{c_choice}]')
        render(board, c_choice, h_choice)
        print('YOU LOSE!')
    else:
        clean()
        render(board, c_choice, h_choice)
        print('DRAW!')

    exit()

In [ ]:
if __name__ == '__main__':
    main()


Choose X or O
Chosen: x
First to start?[y/n]: n
AI turn [O]

---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
|   ||   ||   |
---------------
|   || O ||   |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 6
AI turn [O]

---------------
|   ||   ||   |
---------------
|   || O || X |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
| O ||   ||   |
---------------
|   || O || X |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 9
AI turn [O]

---------------
| O ||   ||   |
---------------
|   || O || X |
---------------
|   ||   || X |
---------------
Human turn [X]

---------------
| O ||   || O |
---------------
|   || O || X |
---------------
|   ||   || X |
---------------
Use numpad (1..9): 8
AI turn [O]

---------------
| O ||   || O |
---------------
|   || O || X |
---------------
|   || X || X |
---------------
AI t